In [ ]:
scores = []
best_embs = None
best_ep = None
prev_best_score = -10
for i in range(0, 29, 2) :
    print('-----------------------------')
    print('Epoch {}'.format(i))
    
    model.load_state_dict(torch.load('data/tests_model_image/model_arcf_split_res50_ep_{}.pth'.format(i)))
    with torch.no_grad():
        pbar = tqdm(val_dl, leave=False)
        embs = []
        for imgs, _ in pbar:
            imgs = val_transforms(imgs.to(device))
            feature = model(imgs)
            embs.append(feature)
        embs = F.normalize(torch.cat(embs, 0))
    scs = [compute_f1(embs, val_df['label_group'], thr) for thr in np.linspace(0.1, 1.3, 22)]
    scores.append((i, scs))
    best = sorted(scs, key=lambda x : x[0], reverse=True)[0][0]
    if best > prev_best_score :
        prev_best_score = best
        best_embs = embs
        best_ep = i